In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
sales = pd.read_csv("C:\\Users\\DHRUBAJIT\\Desktop\\AnalyticsVidhya\\Big Mart Sales\\new_sales_csv.csv")
sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_type_new
0,FDA15,weights2,Low Fat,low,Dairy,249.8092,OUT049,med,High,Tier 1,Supermarket Type1,3735.1380,other
1,DRC01,weights1,Regular,low,Soft Drinks,48.2692,OUT018,low,High,Tier 2,Supermarket Type2,443.4228,Drinks
2,FDN15,weights1,Low Fat,low,Meat,141.6180,OUT049,med,High,Tier 1,Supermarket Type1,2097.2700,other
3,FDX07,weights1,Regular,low,Fruits and Vegetables,182.0950,OUT010,med,High,Tier 2,Grocery Store,732.3800,other
4,NCD19,weights1,Low Fat,low,Household,53.8614,OUT013,med,High,Tier 2,Supermarket Type1,994.7052,other


In [3]:
#deleting unwanted features 
#also deleted Item_Fat_Content because from hypothesis testing,we found that there is no difference in the means.
sales = sales.drop(['Item_Identifier','Item_Type','Outlet_Identifier','Item_Fat_Content'], axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

encoder_list = ['Item_Weight', 'Item_Visibility',
       'Outlet_Establishment_Year', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type', 
       'Item_type_new']

for i in encoder_list:
    sales[i] = le.fit_transform(sales[i])

In [5]:
sales = pd.get_dummies(sales, columns=encoder_list)

In [6]:
#avoiding dummy variable trap
sales = sales.drop(['Item_Weight_0','Item_Visibility_0','Outlet_Establishment_Year_0','Outlet_Size_0',
                   'Outlet_Location_Type_0','Outlet_Type_0','Item_type_new_0'], axis=1)

In [7]:
X = sales.drop(['Item_Outlet_Sales'], axis=1)
y = sales.Item_Outlet_Sales

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
sc = StandardScaler()
XX = sc.fit_transform(X)

#### Test dataset

In [9]:
test = pd.read_csv("C:\\Users\\DHRUBAJIT\\Desktop\\AnalyticsVidhya\\Big Mart Sales\\new_testsales_csv.csv")

item_id = test['Item_Identifier']
out_id = test['Outlet_Identifier']

test = test.drop(['Item_Identifier','Item_Type','Outlet_Identifier','Item_Fat_Content'], axis=1)

for i in encoder_list:
    test[i] = le.fit_transform(test[i])
    
test = pd.get_dummies(test, columns=encoder_list)

test = test.drop(['Item_Weight_0','Item_Visibility_0','Outlet_Establishment_Year_0','Outlet_Size_0',
                   'Outlet_Location_Type_0','Outlet_Type_0','Item_type_new_0'], axis=1)

test = sc.fit_transform(test)

In [77]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def validation_cross(model, x,y):
    model.fit(x, y)

    #Predict training set:
    pred = model.predict(x)

    #Perform cross-validation:
    cv_score = cross_val_score(model, x, y, cv=10, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))

    #Print model report:
    print ("\nModel Report")
    print ("RMSE : %.4g" % np.sqrt(mean_squared_error(y.values, pred)))
    print ("CV Score : Mean - %.4g | Std - %.4g" % (np.mean(cv_score),np.std(cv_score)))
    

def validation_cross1(model, x,y):
    model.fit(x, y)

    #Predict training set:
    pred = model.predict(x)

    #Perform cross-validation:
    cv_score = cross_val_score(model, x, y, cv=10, scoring=None)
    cv_score = np.sqrt(np.abs(cv_score))

    #Print model report:
    print ("\nModel Report")
    print ("RMSE : %.4g" % np.sqrt(mean_squared_error(y.values, pred)))
    print ("CV Score : Mean - %.4g | Std - %.4g" % (np.mean(cv_score),np.std(cv_score)))

### Baseline Model

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
validation_cross(gbr, XX, y)


Model Report
RMSE : 1048
CV Score : Mean - 1090 | Std - 19.42


In [78]:
validation_cross1(gbr, XX, y)


Model Report
RMSE : 1048
CV Score : Mean - 0.768 | Std - 0.0179


#### Model Tuning

In [43]:
# 1. n_estimators

param_test1 = {'n_estimators':range(50,1050,50)}
gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = param_test1, n_jobs=-1, iid=False, cv=10)
gsearch1.fit(XX,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.59199, std: 0.02675, params: {'n_estimators': 50},
  mean: 0.59013, std: 0.02726, params: {'n_estimators': 100},
  mean: 0.58529, std: 0.02745, params: {'n_estimators': 150},
  mean: 0.58101, std: 0.02809, params: {'n_estimators': 200},
  mean: 0.57580, std: 0.02860, params: {'n_estimators': 250},
  mean: 0.57162, std: 0.02833, params: {'n_estimators': 300},
  mean: 0.56776, std: 0.02889, params: {'n_estimators': 350},
  mean: 0.56443, std: 0.03020, params: {'n_estimators': 400},
  mean: 0.56089, std: 0.03073, params: {'n_estimators': 450},
  mean: 0.55874, std: 0.03043, params: {'n_estimators': 500},
  mean: 0.55705, std: 0.03100, params: {'n_estimators': 550},
  mean: 0.55380, std: 0.03222, params: {'n_estimators': 600},
  mean: 0.55107, std: 0.03204, params: {'n_estimators': 650},
  mean: 0.54862, std: 0.03255, params: {'n_estimators': 700},
  mean: 0.54641, std: 0.03320, params: {'n_estimators': 750},
  mean: 0.54407, std: 0.03416, params: {'n_estimators': 800},
  mean: 0

In [79]:
gbr1 = GradientBoostingRegressor(n_estimators = 50)
validation_cross1(gbr1, XX, y)


Model Report
RMSE : 1064
CV Score : Mean - 0.7692 | Std - 0.0175


In [46]:
# 2. max_depth

param_test2 = {'max_depth':range(1,10,1)}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = param_test2,n_jobs=-1,iid=False, cv=10)
gsearch2.fit(XX,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.50768, std: 0.01922, params: {'max_depth': 1},
  mean: 0.58662, std: 0.02593, params: {'max_depth': 2},
  mean: 0.59027, std: 0.02736, params: {'max_depth': 3},
  mean: 0.58352, std: 0.02760, params: {'max_depth': 4},
  mean: 0.57495, std: 0.03059, params: {'max_depth': 5},
  mean: 0.56191, std: 0.03133, params: {'max_depth': 6},
  mean: 0.55268, std: 0.03259, params: {'max_depth': 7},
  mean: 0.53706, std: 0.03367, params: {'max_depth': 8},
  mean: 0.51473, std: 0.03653, params: {'max_depth': 9}],
 {'max_depth': 3},
 0.59026669882689153)

In [80]:
gbr2 = GradientBoostingRegressor(max_depth = 3)
validation_cross1(gbr2, XX, y)


Model Report
RMSE : 1048
CV Score : Mean - 0.7679 | Std - 0.01779


In [50]:
# 3. min_samples_split & min_samples_leaf

param_test3 = {'min_samples_split':range(2,20,1), 'min_samples_leaf':range(1,20,1)}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = param_test3,n_jobs=-1,iid=False, cv=10)
gsearch3.fit(XX,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.59018, std: 0.02732, params: {'min_samples_split': 2, 'min_samples_leaf': 1},
  mean: 0.59020, std: 0.02718, params: {'min_samples_split': 3, 'min_samples_leaf': 1},
  mean: 0.59025, std: 0.02710, params: {'min_samples_split': 4, 'min_samples_leaf': 1},
  mean: 0.58952, std: 0.02678, params: {'min_samples_split': 5, 'min_samples_leaf': 1},
  mean: 0.58968, std: 0.02724, params: {'min_samples_split': 6, 'min_samples_leaf': 1},
  mean: 0.58983, std: 0.02696, params: {'min_samples_split': 7, 'min_samples_leaf': 1},
  mean: 0.59018, std: 0.02715, params: {'min_samples_split': 8, 'min_samples_leaf': 1},
  mean: 0.59022, std: 0.02709, params: {'min_samples_split': 9, 'min_samples_leaf': 1},
  mean: 0.58969, std: 0.02715, params: {'min_samples_split': 10, 'min_samples_leaf': 1},
  mean: 0.59030, std: 0.02678, params: {'min_samples_split': 11, 'min_samples_leaf': 1},
  mean: 0.59022, std: 0.02684, params: {'min_samples_split': 12, 'min_samples_leaf': 1},
  mean: 0.59033, std: 0.02670

In [81]:
gbr3 = GradientBoostingRegressor(min_samples_split = 19, min_samples_leaf = 9)
validation_cross1(gbr3, XX, y)


Model Report
RMSE : 1051
CV Score : Mean - 0.7706 | Std - 0.01783


In [53]:
# 4. max_features

param_test4 = {'max_features':range(1,10,1)}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(),param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(XX,y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.57044, std: 0.02599, params: {'max_features': 1},
  mean: 0.58468, std: 0.02772, params: {'max_features': 2},
  mean: 0.59249, std: 0.02749, params: {'max_features': 3},
  mean: 0.59306, std: 0.02692, params: {'max_features': 4},
  mean: 0.59371, std: 0.02590, params: {'max_features': 5},
  mean: 0.59185, std: 0.02729, params: {'max_features': 6},
  mean: 0.59207, std: 0.02697, params: {'max_features': 7},
  mean: 0.59183, std: 0.02675, params: {'max_features': 8},
  mean: 0.59118, std: 0.02599, params: {'max_features': 9}],
 {'max_features': 5},
 0.59370527977650922)

In [82]:
gbr4 = GradientBoostingRegressor(max_features = 5)
validation_cross1(gbr4, XX, y)


Model Report
RMSE : 1055
CV Score : Mean - 0.7703 | Std - 0.01748


In [57]:
# 4. max_features - strings

param_test4 = {'max_features':['auto','sqrt','log2']}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(),param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(XX,y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.59017, std: 0.02731, params: {'max_features': 'auto'},
  mean: 0.59238, std: 0.02754, params: {'max_features': 'sqrt'},
  mean: 0.59205, std: 0.02561, params: {'max_features': 'log2'}],
 {'max_features': 'sqrt'},
 0.59238410436769617)

In [83]:
gbr4 = GradientBoostingRegressor(max_features = 'sqrt')
validation_cross1(gbr4, XX, y)


Model Report
RMSE : 1063
CV Score : Mean - 0.7698 | Std - 0.01797


In [86]:
# 5. subsample

param_test5 = {'subsample': [0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0]}
gsearch5 = GridSearchCV(estimator = GradientBoostingRegressor(),param_grid = param_test5,n_jobs=-1,iid=False, cv=10)
gsearch5.fit(XX,y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.58993, std: 0.02692, params: {'subsample': 0.6},
  mean: 0.59002, std: 0.02705, params: {'subsample': 0.65},
  mean: 0.58874, std: 0.02803, params: {'subsample': 0.7},
  mean: 0.59038, std: 0.02813, params: {'subsample': 0.75},
  mean: 0.59047, std: 0.02746, params: {'subsample': 0.8},
  mean: 0.59076, std: 0.02533, params: {'subsample': 0.85},
  mean: 0.59048, std: 0.02601, params: {'subsample': 0.9},
  mean: 0.59134, std: 0.02674, params: {'subsample': 0.95},
  mean: 0.59030, std: 0.02737, params: {'subsample': 1.0}],
 {'subsample': 0.95},
 0.59133595051823284)

In [87]:
gbr5 = GradientBoostingRegressor(subsample = 0.95)
validation_cross1(gbr5, XX, y)


Model Report
RMSE : 1044
CV Score : Mean - 0.7675 | Std - 0.01837


In [66]:
# 6. learning_rate

param_test6 = {'learning_rate':[0.0001,0.001,0.001,0.005,0.01,0.05,0.1,0.2,0.3,0.4]}
gsearch6 = GridSearchCV(estimator = GradientBoostingRegressor(),param_grid = param_test6,n_jobs=-1,iid=False, cv=10)
gsearch6.fit(XX,y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.00858, std: 0.00082, params: {'learning_rate': 0.0001},
  mean: 0.08386, std: 0.00222, params: {'learning_rate': 0.001},
  mean: 0.08386, std: 0.00222, params: {'learning_rate': 0.001},
  mean: 0.29986, std: 0.00791, params: {'learning_rate': 0.005},
  mean: 0.41970, std: 0.01404, params: {'learning_rate': 0.01},
  mean: 0.59204, std: 0.02630, params: {'learning_rate': 0.05},
  mean: 0.59015, std: 0.02728, params: {'learning_rate': 0.1},
  mean: 0.57859, std: 0.02916, params: {'learning_rate': 0.2},
  mean: 0.57076, std: 0.02512, params: {'learning_rate': 0.3},
  mean: 0.55848, std: 0.03161, params: {'learning_rate': 0.4}],
 {'learning_rate': 0.05},
 0.59204208778387091)

In [67]:
gbr6 = GradientBoostingRegressor(learning_rate= 0.05)
validation_cross(gbr6, XX, y)


Model Report
RMSE : 1065
CV Score : Mean - 1088 | Std - 20.54


In [70]:

param_test7 = {'loss':['ls','lad','huber','quantile']}
gsearch7 = GridSearchCV(estimator = GradientBoostingRegressor(),param_grid = param_test7,n_jobs=-1,iid=False, cv=10)
gsearch7.fit(XX,y)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

C:\Users\DHRUBAJIT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.59017, std: 0.02723, params: {'loss': 'ls'},
  mean: 0.59182, std: 0.02190, params: {'loss': 'lad'},
  mean: 0.58998, std: 0.02626, params: {'loss': 'huber'},
  mean: -0.09519, std: 0.14511, params: {'loss': 'quantile'}],
 {'loss': 'lad'},
 0.59181739762075936)

In [73]:
gbr7 = GradientBoostingRegressor(loss= 'lad')
validation_cross(gbr7, XX, y)


Model Report
RMSE : 1069
CV Score : Mean - 1089 | Std - 20.01
